In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import glob
import time 
import numpy as np
from scipy import optimize
import numba as nb
from mwc_class import getMVWs
from mwc_functions import *
from optimization_functions import *

In [ ]:
def country_name_from_file(file_path):
    #gets the name of the country from the currently 'inspected' file
    #helper function for next cell
    file = os.path.basename(file_path)
    country_name, _ = os.path.splitext(file)
    return country_name.capitalize()



# Loop for all countries:

In [ ]:
""" folder = 'data/'

start_time = time.time()

for csv_file in glob.glob(os.path.join(folder, '*.csv')):
    country_name = country_name_from_file(csv_file)
    country_data = getMVWs(f'{country_name}.csv', name=country_name, save_results=True)
    
    prelims = country_data.preliminaries()
    country_MIW = country_data.minimal_voting_weights_pipeline()

end_time = time.time()
total_time = end_time - start_time
print(f"Total time: {total_time:.2f} seconds") """

# Test for Belgium 

In [ ]:
spain = getMVWs('belgium.csv',name='Belgium',save_results=False,verify_mwcs=False)

In [ ]:
belgian_prelims = spain.preliminaries()

In [ ]:
transformed_dataframe = belgian_prelims.get("transformed_dataframe")
parties_in_year = belgian_prelims.get("parties_in_year")
n_in_year = belgian_prelims.get("n_in_year")
totalseats_in_year = belgian_prelims.get("totalseats_in_year")
coalition_dict = belgian_prelims.get("coalition_dict")
winning_coal_dict = belgian_prelims.get("winning_coal_dict")
minimal_winning_coalitions = belgian_prelims.get("minimal_winning_coalitions")
maximal_losing_coalitions = belgian_prelims.get("maximal_losing_coalitions")
unique_tying_coalitions = belgian_prelims.get("unique_tying_coalitions")

In [ ]:
start_time=time.time()
belgian_miw = spain.minimal_voting_weights_pipeline()
end_time=time.time()
runtime_noverify=end_time-start_time
print(runtime_noverify)

In [ ]:
spain.saveresults=True
spain.save_pipeline()

results look fishy

In [2]:
austria_Data = getMVWs('austria.csv', name='austria', save_results=True,verify_mwcs=True)
    
prelims = austria_Data.preliminaries()
country_MIW = austria_Data.minimal_voting_weights_pipeline()


but seem to work for smaller parliaments

lets check belgium again, with adding unique tying constraints, first: baseline 

In [ ]:
relevant_coals=combine_dicts(spain.minimal_winning_coalitions,spain.maximal_losing_coalitions)
df_for_belgium = create_all_year_dfs(relevant_coals,parties_in_year)
belgian_mwc_constraints = generate_constraints_df(df_for_belgium['2014'])
belgian_lin_cons= get_lin_cons(belgian_mwc_constraints)
beligan_mvw=get_min_vote_weights('2014',n_in_year,belgian_lin_cons)
print(beligan_mvw.x)
print(sum(beligan_mvw.x))

trying to add more structure to belgian data

In [ ]:
def append_tying_coals_to_cons(year,constraints_df, unique_tying,parties_in_year):
    ##takes in constraints_df and adds unique-tying constraints to it 
    constraints_copy=constraints_df.copy()
    constraint_index = len(constraints_copy)  # make sure to add to the bottom 
    
    for (yr, (coalition, complementary_coal)) in unique_tying.keys():
        if yr==year: #make sure to only add contraints of the correct year
            constraint_row = {f'w_{party}': 0 for party in parties_in_year[year]}  # Initialize all party weights to 0 #create a dict for every row - first set keys to column names and attribute all of them value 0 
            for party in coalition.split('+'): #update value dapending on unique-tying dict 
                constraint_row[f'w_{party}'] = 1
            for party in complementary_coal.split('+'):
                constraint_row[f'w_{party}'] = -1
            # Add to df
            constraints_copy.loc[constraint_index] = constraint_row
            constraint_index += 1

    return constraints_copy

In [ ]:
new_constraints = append_tying_coals_to_cons('2014',belgian_mwc_constraints,unique_tying_coalitions,parties_in_year)

need to account for differnet constraint form of unique tying coals "(tying_coal)-(complement)"=0

In [ ]:
def get_new_lin_cons(year,constraints_df,unique_tying_coalitions):
    # now also incorporates the fact the the unique tying coals have to be exactly 0    
    A = constraints_df.to_numpy()
    lbnd = np.zeros(len(constraints_df)) # non-negativity constraints 
    lbnd[constraints_df.shape[1]:] = 1 #set all remaining lower bounds to 1
    upbnd = np.full(len(constraints_df), np.inf) #no upper bound for most constraints 
    #get # of unique tying coals 
    nr_of_ties = sum(1 for key in unique_tying_coalitions.keys() if key[0] == year) #count how many keys there are for the year 
    upbnd[-nr_of_ties:]=0 
    lbnd[-nr_of_ties:]=0

    lin_cons = LinearConstraint(A, lbnd, upbnd)

    return lin_cons

In [ ]:
new_lin_cons = get_new_lin_cons('2014',new_constraints,unique_tying_coalitions)

In [ ]:
get_min_vote_weights('2014',n_in_year,new_lin_cons)

In [ ]:
print(beligan_mvw.x)

adding more constraints does not change the result 

# double belgian parliament and test for result 

In [ ]:
belgium = read_csv_to_dataframe('belgium.csv')
df = transform_and_sort_dataframe(belgium)
df['# of Seats']=df['# of Seats']*5
parties_in_year, totalseats_in_year, n_in_year= variables_by_year(df)
coalition_dict = coalition_combinatorics_generator(df,parties_in_year)

pipeline manual

In [ ]:
begin = time.time()
winning_coal_dict=win_coals(coalition_dict, totalseats_in_year)
min_win_coal_dict=min_winning_coals(winning_coal_dict)
maximal_losing_dict=max_loosing_coals(winning_coal_dict, parties_in_year)
combined_dict=combine_dicts(min_win_coal_dict, maximal_losing_dict)
all_year_dfs=create_all_year_dfs(combined_dict, parties_in_year)
constraints_df=generate_constraints_df(all_year_dfs['2014'])
lin_con=get_lin_cons(constraints_df)
min_weights=get_min_vote_weights('2014',n_in_year,lin_con)
end = time.time()
duration = end-begin 


In [ ]:
results=mvw_to_parties('2014',min_weights,parties_in_year)
(results)

also, same result, will believe it to be correct for now 

## Now test for really big pariliaments 

# spain 2019 - n = 18 

In [ ]:
start_time=time.time()
spain = getMVWs('spain.csv',name='Spain',save_results=True,verify_mwcs=False)
spain_prelims = spain.preliminaries()

middle_time = time.time()
first_timer = middle_time-start_time
print(first_timer)

spain_miw = spain.minimal_voting_weights_pipeline()

end_time=time.time()
runtime_noverify=end_time-start_time
print(runtime_noverify)

kinda cracked, ngl :D

next hickup: israel

In [ ]:
start_time = time.time()

israel_data = getMVWs('israel.csv', name='Israel', save_results=False,find_errors=True)
prelims = israel_data.preliminaries()
prelim_time = israel_data.time - start_time
print(prelim_time)
country_MIW = israel_data.minimal_voting_weights_pipeline()
pipeline_time = israel_data.time - prelim_time
print(pipeline_time)
end_time = time.time()
total_time = end_time - start_time
print(f"Total time: {total_time:.2f} seconds") 

investigate israel 1999:

In [ ]:
israel_data = getMVWs('israel.csv',name='israel',save_results=False,verify_mwcs=False,find_errors=True)

In [ ]:
israel_prelims = israel_data.preliminaries()

In [ ]:
transformed_dataframe = israel_prelims.get("transformed_dataframe")
parties_in_year = israel_prelims.get("parties_in_year")
n_in_year = israel_prelims.get("n_in_year")
totalseats_in_year = israel_prelims.get("totalseats_in_year")
coalition_dict = israel_prelims.get("coalition_dict")
winning_coal_dict = israel_prelims.get("winning_coal_dict")
minimal_winning_coalitions = israel_prelims.get("minimal_winning_coalitions")
maximal_losing_coalitions = israel_prelims.get("maximal_losing_coalitions")
unique_tying_coalitions = israel_prelims.get("unique_tying_coalitions")

In [ ]:
begin = time.time()
combined_dict=combine_dicts(minimal_winning_coalitions, maximal_losing_coalitions)
all_year_dfs= create_all_year_dfs(combined_dict,parties_in_year)
end = time.time()
duration = end-begin 
print(duration)


In [ ]:

def constraint_generation_with_timer(df):


    parties = df.columns[:-1] 
    constraints_df = pd.DataFrame(columns=parties) #create new empty df with weights as columns  
    constraint_index = 0 # to allow .loc[] instead of .append, basically counts constraints  

    winning_coalitions = df[df['Winning'] == 1] # Split original DataFrame into winning and losing coalitions
    losing_coalitions = df[df['Winning'] == 0]

    # non-negativity constraints
    for party in parties:
        non_negative_constraint = {p: 1 if p == party else 0 for p in parties} # puts a 1 where respective p is equal to the party and 0 else 
        constraints_df.loc[constraint_index] = non_negative_constraint #adds rows for every constraint
        constraint_index += 1 #update index counter

    start_time = time.time()
    last_stamp = constraint_index 
    # w(S)>w(R) constraints
    for _, winning_row in winning_coalitions.iterrows(): #outerloop with the winning coals
        for _, losing_row in losing_coalitions.iterrows(): #inner loop with the losing coals
            
            # Create a dict for the constraint (party,value) where value \in {-1,0,1}
            constraint_row = {}
            for party in parties: #calculate difference between the winning and the losing coal
                constraint_row[party] = winning_row[party] - losing_row[party] #checks the value of the party in the winning coal (either 1 if present or 0 if not) and the losing coal. Example S={CDU,SPD,B90}, R={SPD,B90,FDP} then CDU=1-0, SPD=1-1, B90=1-1, FDP=0-1. Resulting constraint is w_cdu-w_fdp>0 

            # Add the constraint to the DataFrame
            constraints_df.loc[constraint_index] = constraint_row
            constraint_index += 1
            if constraint_index % 1000 == 0:
                    current_time = time.time()
                    print(f"Took {current_time - start_time:.2f} seconds to write constraints {last_stamp} to {constraint_index}")
                    start_time = current_time 
                    last_stamp = constraint_index
    # Renaming 
    constraints_df.columns = [f'w_{party}' for party in parties]

    return constraints_df



In [ ]:
df_1999=all_year_dfs['1999']

In [ ]:
begin = time.time()
constraint_generation_with_timer(df_1999)
end = time.time()

duration = end-begin 
print(duration)

need a more efficient constraints generator: 

In [18]:
def generate_eff_cons(df):
    
    parties = df.columns[:-1].tolist()
    n = len(parties) 

    
    winning_coalitions = df[df['Winning'] == 1] 
    losing_coalitions = df[df['Winning'] == 0]
    
    winning_array = winning_coalitions.drop(columns=['Winning']).to_numpy()
    losing_array = losing_coalitions.drop(columns=['Winning']).to_numpy()
    
    n_constraints = len(winning_array) * len(losing_array) + n
    constraints_array = np.zeros((n_constraints, n),dtype='int') #store constraints in an array, as its much faster than a df, initialize 0´s everywhere
    
    # Non-negativity constraints
    for i, party in enumerate(parties): #enum creates (1,party_1),(2,party_2),... elements thus needs to call i,party even if party is never accessed 
        constraints_array[i, i] = 1 # write a 1 where party=party 
    
    # w(S) > w(R) constraints
    constraint_index = n  # Start after non-negativity cons
    for win_row in winning_array:
        for lose_row in losing_array:
            # #calculate difference between the winning and the losing coal
            constraints_array[constraint_index] = win_row - lose_row #checks the value of the party in the winning coal (either 1 if present or 0 if not) and the losing coal. Example S={CDU,SPD,B90}, R={SPD,B90,FDP} then CDU=1-0, SPD=1-1, B90=1-1, FDP=0-1. Resulting constraint is w_cdu-w_fdp>0 
            constraint_index += 1
            
            
    # array to original df 
    constraints_df = pd.DataFrame(constraints_array, columns=[f'w_{party}' for party in parties])
    
    return constraints_df


In [ ]:
begin = time.time()
cons_1999= generate_eff_cons(df_1999)
end = time.time()

check whether new dfs are correct: 

In [12]:
old_data_frames={}
for year, df in austria_Data.all_constraints.items():
    old_data_frames[year] = df  

In [19]:
new_data_frames = {}
for year,df in austria_Data.all_dfs.items(): 
    new_data_frames[year]=generate_eff_cons(austria_Data.all_dfs[year]) 

In [20]:
test_dict ={}
for year,df in old_data_frames.items():
    is_identical = new_data_frames[year].equals(old_data_frames[year])
    test_dict[year]=is_identical
print(test_dict)

{'1994': False, '1995': False, '1999': False, '2002': False, '2006': False, '2008': False, '2013': False, '2017': False, '2019': False}


In [23]:
len(old_data_frames['1994'])

14

In [24]:
len(new_data_frames['1994'])

14

In [25]:
old_data_frames['1994']==new_data_frames['1994']

,w_FPÖ,w_GA,w_LIF,w_SPÖ,w_ÖVP
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
5,True,True,True,True,True
6,True,True,True,True,True
7,True,True,True,True,True
8,True,True,True,True,True
9,True,True,True,True,True
